# Track military vehicles in video inputs

In this notebook, we'll use the model that we trained with our custom dataset to track military vehicles in videos. We'll use the tracking algorithms provided in the [YOLO](https://github.com/ultralytics/ultralytics) library.

We assume in this notebook that notebooks [01_Prepare](./01_Prepare.ipynb) and [02_Train](./02_Train.ipynb) have already been run in order to train a Yolov8 model on a military vehicle dataset.

### Setup

To start, check GPU support.

In [1]:
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.3.1+cu121 (NVIDIA RTX A6000)


### Run inference on the video

We run our detection model on a sample video provided in the `resources` directory, saving the output to disk.

In [2]:
from pathlib import Path
from adomvi.yolo.yolo import track

model_path = Path() / "runs/detect/train/weights/best.pt"
video_path = Path() / "../resources/test/apc1.mp4"
results_track = track(model_path, source=video_path, tracker="bytetrack.yaml")



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/511) /home/jrenault/workspace/adomvi2/notebooks/../resources/test/apc1.mp4: 384x640 1 tank, 138.4ms
video 1/1 (frame 2/511) /home/jrenault/workspace/adomvi2/notebooks/../resources/test/apc1.mp4: 384x640 1 tank, 6.8ms
video 1/1 (frame 3/511) /home/jrenault/workspace/adomvi2/notebooks/../resources/test/apc1.mp4: 384x640 1 tank, 5.8ms
video 1/1 (frame 4/511) /home/jrenault/workspace/adomvi2/notebooks/../resources/test/apc1.mp4: 384x640 

In [3]:
# Load the path of the track model results

results_track_dir = Path(results_track[0].save_dir)

### Display the result video

In order to display the resulting video in a notebook, we compress the video file to a format supported by HTML (mpeg4 (native) -> h264 (libx264))

In [4]:
from adomvi.utils import convert_video_avi_to_mp4

input_video_path = Path() / results_track_dir / "apc1.avi"
output_video_path = Path() / results_track_dir / "apc1.mp4"
convert_video_avi_to_mp4(input_video_path, output_video_path)

Moviepy - Building video runs/detect/track2/apc1.mp4.
Moviepy - Writing video runs/detect/track2/apc1.mp4



Moviepy - Done !
Moviepy - video ready runs/detect/track2/apc1.mp4


We can then display it in a frame.

In [5]:
import IPython.display as display
from base64 import b64encode

video_url = (f"data:video/mp4;base64,{b64encode(output_video_path.read_bytes()).decode()}")
display.HTML(
    f"""
    <video controls width="640">
        <source src="{video_url}" type="video/mp4">
    </video>
    """
)